# Llama Index Demo 및 발표
### 디지털애널리틱스융합협동과정 2023311??? 최민철 <br> 디지털애널리틱스융합협동과정 2023311561 이정환

# Basic Knowledge
- **LLM에서 학습되지 않은 데이터 : 사내 데이터, 개인 데이터**  
- **추가 학습 없이 : 데이터 참조 형식**
- **다양한 데이터 형식 : pdf 등 파일형식, Youtube와 같은 웹 서비스**

## RAG(Retrieval Augmented Generation) vs FineTuning
<!-- ![rag.png](./image/RAG.png) -->
<!-- ![finetuning.png](./image/finetuning.png) -->
![rag_vs_ft.png](./image/rag_vs_finetuning.png)

## RAG 5 단계
![rag_five_stage.png](./image/rag_five_stage.png)

- **로딩(Loading)**: 데이터를 소스에서 파이프라인으로 가져오는 단계. (LlamaHub)
- **인덱싱(Indexing)**: 데이터 쿼리를 가능하게 하는 데이터 구조를 생성하는 과정. 주로 벡터 임베딩 생성.
- **저장(Storing)**: 인덱스와 다른 메타데이터 저장. 매번 인덱싱할 필요 없음.
- **쿼리(Querying)**: LLM과 LlamaIndex 데이터 구조를 사용하여 쿼리 수행.
- **평가(Evaluation)**: 쿼리에 대한 응답의 정확성, 충실도, 속도 측정.

## 종합적인 사용 사례
- **쿼리 엔진**: 데이터에 대해 자연어 쿼리를 통해 질문하여 참조 컨텍스트를 LLM에 전달하는 엔드 투 엔드 파이프라인
- **채팅 엔진**: 데이터와의 대화를 위한 엔드 투 엔드 파이프라인
- **에이전트**: LLM으로 구동되는 자동화된 의사 결정자


<br><br>
<참고자료>  
https://medium.com/neo4j/knowledge-graphs-llms-fine-tuning-vs-retrieval-augmented-generation-30e875d63a35  
https://gpt-index.readthedocs.io/en/stable/  
https://llamahub.ai/

# Llama Index VS Langchain

- **라마인덱스는 랭체인 라이브러리로 구축된 애플리케이션**
- **랭체인은 조금 더 전반적인 기능들을 다룰 수 있음. loading, processing, indexing, LLM과의 interaction 등. 이로써 사용자는 사용자의 의도에 따라 커스터마이징에 유리함**
- **반면, 라마인덱스는 search and retrieval에 초점을 맞춘 애플리케이션으로 거대한 양의 데이터를 처리하기에 좋음**

# Demo

# requirements settings

In [ ]:
# !git clone https://github.com/jerryjliu/llama_index.git
# %pip install poetry
# %cd llama_index
# %poetry install
# %pip install -U llama-index
# %pip install -U langchain
# %pip install sentence-transformers
# %pip install accelerate
# %pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
# %pip install transformers==4.34.0
# %pip install faiss-cpu
# %pip install lamma-hub
# %pip install youtube_transcript_api

In [2]:
#로그 설정 & API Key 설정
import os
import logging
import sys
from dotenv import load_dotenv
import os 

# load .env
load_dotenv()
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

# 데이터 로드

사용하고 싶은 데이터(문서) 로드

In [3]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader("data").load_data()

DEBUG:llama_index.readers.file.base:> [SimpleDirectoryReader] Total files added: 7


# LLM


- **Default : OpenAI의 chatgpt text-davinci-003**
- **For Demo : gpt-3.5-turbo**
- **OpenAI의 다른 모델 (e.g. gpt-3.5-turbo 등), Custom Model, huggingface 모델도 사용 가능**



In [4]:
from llama_index import LLMPredictor
from langchain.chat_models import ChatOpenAI


llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo"))


# from langchain.llms.base import LLM
# from typing import Optional, List, Mapping, Any
# from transformers import pipeline

# # setting
# context_window = 2048
# num_output = 256
# model_name = 'beomi/KoAlpaca-llama-1-7b'
# pipe = pipeline("text-generation", model=model_name)

# class CustomLLM(LLM):
#     def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
#         prompt_length = len(prompt)
#         response = pipeline(prompt, max_new_tokens=num_output)[0]["generated_text"]

#         # only return newly generated tokens
#         return response[prompt_length:]

#     @property
#     def _identifying_params(self) -> Mapping[str, Any]:
#         return {"name_of_model": model_name}

#     @property
#     def _llm_type(self) -> str:
#         return "custom"

    
# # define our LLM
# llm_predictor = LLMPredictor(llm=CustomLLM())

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/sean_forhim/opt/anaconda3/envs/llama_index/lib/python3.9/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/sean_forhim/opt/anaconda3/envs/llama_index/lib/python3.9/site-packages/certifi/cacert.pem'


# Embedding

- **Default : OpenAI의 text-embedding-ada-002**
- **For Demo : Huggingface (bongsoo/moco-sentencedistilbertV2.1)**
- **custom model, huggingface 모델이 사용 가능**

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding


embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name='bongsoo/moco-sentencedistilbertV2.1'))

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: bongsoo/moco-sentencedistilbertV2.1
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


# DataBase

- **Default : GPTVectorStoreIndex (Python Dict)**
- **For Demo : Faiss**
- **Vector DataBase : Faiss, Qdrant, Chroma, Milvus (이상 패키지로 제공하는 데이터베이스), Pinecone, Weaviate (이상 클라우드 서비스로 제공하는 데이터베이스)**

In [6]:
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss


# faiss의 indexing : IndexFlatL2, IndexFlatIP, IndexIVFFlat
faiss_index = faiss.IndexFlatL2(768)
vector_store = FaissVectorStore(faiss_index=faiss_index)

INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.


# Index
- **앞선 모듈 객체화 이후 Index 생성**

In [7]:
from llama_index import GPTVectorStoreIndex, ServiceContext, StorageContext


storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor,embed_model=embed_model)
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context, storage_context=storage_context)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): openaipublic.blob.core.windows.net:443
DEBUG:urllib3.connectionpool:https://openaipublic.blob.core.windows.net:443 "GET /encodings/cl100k_base.tiktoken HTTP/1.1" 200 1681126
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제1장: 데이터 프론트

밤이 되면 반짝이는 네오 도쿄. 고층 빌딩이 늘어서고, 네온...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제2장: 울프 코퍼레이션의 함정

미코는 목적지인 술집 '할머니의 집'으로 향하는 길...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제3장: 배신과 재회

술집 '할머니의 집'에서 미코는 데이터를 받을 사람인 료를 기...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제4장: 울프 코퍼레이션의 붕괴

미코와 료는 해커 집단과 함께 울프 코퍼레이션에 대...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제5장: 결전의 순간

미코와 료는 마침내 울프 코퍼레이션의 최상층에 도착해 CEO인...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제6장: 진실의 해방

미코는 울프 박사의 약점을 파고들어 그를 쓰러뜨리는데 성공한다...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 제7장: 새로운 시작

울프 코퍼레이션이 무너진 후, 미코와 료는 서로의 과거를 용서...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

# Query Engine
- **From Index : 사용자의 쿼리와 입력된 정보**
- **From LLM : 해당 정보와 입력을 바탕으로 응답 생성**

In [8]:
query_engine = index.as_query_engine()

In [9]:
print(query_engine.query("미코의 성격은?"))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:llama_index.indices.utils:> Top 2 nodes:
> [Node 5] [Similarity score:             47.2367] 제6장: 진실의 해방

미코는 울프 박사의 약점을 파고들어 그를 쓰러뜨리는데 성공한다. 그리고 해커 집단과 함께 울프 코퍼레이션의 악행을 세상에 공개하고 시민들을 해방시킨다....
> [Node 4] [Similarity score:             48.4738] 제5장: 결전의 순간

미코와 료는 마침내 울프 코퍼레이션의 최상층에 도착해 CEO인 교활한 울프 박사와 대면한다. 울프 박사는 시민을 지배하려는 사악한 야망을 드러내며 자신...
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."}, {'role': 'user', 'content': 'Context information is below.\n---------------------\nfile_path: data/akazukin6.txt\n\n제6장: 진실의 해

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 27 Nov 2023 13:23:05 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-allow-origin', b'*'), (b'Cache-Control', b'no-cache, must-revalidate'), (b'openai-model', b'gpt-3.5-turbo-0613'), (b'openai-organization', b'user-lj8qeahdj7njstnyft5lsza9'), (b'openai-processing-ms', b'7870'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'200'), (b'x-ratelimit-limit-tokens', b'40000'), (b'x-ratelimit-limit-tokens_usage_based', b'40000'), (b'x-ratelimit-remaining-requests', b'199'), (b'x-ratelimit-remaining-tokens', b'39654'), (b'x-ratelimit-remaining-tokens_usage_based', b'39654'), (b'x-ratelimit-reset-requests', b'7m12s'), (b'x-ratelimit-reset-tokens', b'519ms'), (b'x-ratelimit-reset-tokens_usage_based', b'519ms

In [10]:
print(query_engine.query("울프 코퍼레이션의 CEO의 이름은?"))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:llama_index.indices.utils:> Top 2 nodes:
> [Node 4] [Similarity score:             45.9988] 제5장: 결전의 순간

미코와 료는 마침내 울프 코퍼레이션의 최상층에 도착해 CEO인 교활한 울프 박사와 대면한다. 울프 박사는 시민을 지배하려는 사악한 야망을 드러내며 자신...
> [Node 1] [Similarity score:             47.3819] 제2장: 울프 코퍼레이션의 함정

미코는 목적지인 술집 '할머니의 집'으로 향하는 길에 울프 코퍼레이션의 요원들에게 쫓기게 된다. 그들은 '빨간 망토'라는 데이터 카우리아에 ...
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."}, {'role': 'user', 'content': "Context information is below.\n---------------------\nfile_path: data/akazukin5.txt\n\n제5장: 결전의 순

# Llama Hub

- **위키피디아, 구글 드라이브, 구글 독스, gmail, 노션, spotify, youtube 등**

## Youtube Transcript

In [11]:
from llama_index import download_loader

YoutubeTranscriptReader = download_loader("YoutubeTranscriptReader")
loader = YoutubeTranscriptReader()
yt_transcript = loader.load_data(ytlinks=["https://youtu.be/zPsiPvNod08?si=3pxE0QzthzMI_dEd"])

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.youtube.com:443
DEBUG:urllib3.connectionpool:https://www.youtube.com:443 "GET /watch?v=zPsiPvNod08 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.youtube.com:443
DEBUG:urllib3.connectionpool:https://www.youtube.com:443 "GET /api/timedtext?v=zPsiPvNod08&ei=RphkZcP3D7ux1d8PqNGv-A8&caps=asr&opi=112496729&xoaf=5&hl=ko&ip=0.0.0.0&ipbits=0&expire=1701116598&sparams=ip,ipbits,expire,v,ei,caps,opi,xoaf&signature=4018A144474A52A385EF01E9F70F24510B2EACB5.10BB526B4594A72C7FCDBA18D81FA2442594BEDC&key=yt8&lang=en HTTP/1.1" 200 None


In [12]:
index1 = GPTVectorStoreIndex.from_documents(yt_transcript)
query_engine1 = index1.as_query_engine()

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/sean_forhim/opt/anaconda3/envs/llama_index/lib/python3.9/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/sean_forhim/opt/anaconda3/envs/llama_index/lib/python3.9/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/sean_forhim/opt/anaconda3/envs/llama_index/lib/python3.9/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/sean_forhim/opt/anaconda3/envs/llama_index/lib/python3.9/site-packages/certifi/cacert.pem'
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: Oh, great, great
I came to this place a lot
wit...
DEBUG:llama_index.no

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 27 Nov 2023 13:23:21 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-allow-origin', b'*'), (b'openai-organization', b'user-lj8qeahdj7njstnyft5lsza9'), (b'openai-processing-ms', b'57'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'200'), (b'x-ratelimit-limit-tokens', b'150000'), (b'x-ratelimit-remaining-requests', b'199'), (b'x-ratelimit-remaining-tokens', b'147590'), (b'x-ratelimit-reset-requests', b'7m12s'), (b'x-ratelimit-reset-tokens', b'964ms'), (b'x-request-id', b'5984c492d74f2d93700c9c1953c33844'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=rr0Q2VT8Lfujid4IcXNFN1U3ONVQ7XJo5JP_p7ziH38-1701091401-0-AXqi28szvoh7ystUNGxaw7QKA3zOd3PxLNq/2jiyERlm06meYvtPZ/ZIhn344PEqi3rsdS8ezQMlF5

In [13]:
print(query_engine1.query("이 동영상에서 민수가 가장 먹고 싶어 하는 음식이 무엇인가요?"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x7fea69bf3820>, 'json_data': {'input': ['이 동영상에서 민수가 가장 먹고 싶어 하는 음식이 무엇인가요?'], 'model': <OpenAIEmbeddingModeModel.TEXT_EMBED_ADA_002: 'text-embedding-ada-002'>, 'encoding_format': 'base64'}}
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 27 Nov 2023 13:23:22 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-allow-origin', b'*'), (b'openai-organizat

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Mon, 27 Nov 2023 13:23:25 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-allow-origin', b'*'), (b'Cache-Control', b'no-cache, must-revalidate'), (b'openai-model', b'gpt-3.5-turbo-0613'), (b'openai-organization', b'user-lj8qeahdj7njstnyft5lsza9'), (b'openai-processing-ms', b'3124'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'200'), (b'x-ratelimit-limit-tokens', b'40000'), (b'x-ratelimit-limit-tokens_usage_based', b'40000'), (b'x-ratelimit-remaining-requests', b'197'), (b'x-ratelimit-remaining-tokens', b'38157'), (b'x-ratelimit-remaining-tokens_usage_based', b'38157'), (b'x-ratelimit-reset-requests', b'21m10.899s'), (b'x-ratelimit-reset-tokens', b'2.764s'), (b'x-ratelimit-reset-tokens_usage_based', b